In [19]:
import pandas as pd

orders_df = pd.read_csv("orders.csv")


In [20]:
users_df = pd.read_json("users.json")


In [21]:
import sqlite3

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants_df = pd.read_sql_query("SELECT * FROM restaurants;", conn)


In [22]:
orders_users_df = orders_df.merge(
    users_df,
    on="user_id",
    how="inner"
)


In [23]:
final_df = orders_users_df.merge(
    restaurants_df,
    on="restaurant_id",
    how="inner"
)


In [24]:
# orders + users (left join)
orders_users_df = orders_df.merge(
    users_df,
    on="user_id",
    how="left"
)

# merge with restaurants (left join)
final_df = orders_users_df.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"
)


In [25]:
# Filter only Gold member orders
gold_df = final_df[final_df['membership'] == 'Gold']

# Group by city and sum total_amount
city_revenue = (
    gold_df.groupby('city')['total_amount']
    .sum()
    .reset_index()
    .sort_values(by='total_amount', ascending=False)
)

print(city_revenue)

# Top city
top_city = city_revenue.iloc[0]
print("City with highest revenue from Gold members:", top_city['city'])
print("Total revenue:", top_city['total_amount'])


        city  total_amount
1    Chennai    1080909.79
3       Pune    1003012.32
0  Bangalore     994702.59
2  Hyderabad     896740.19
City with highest revenue from Gold members: Chennai
Total revenue: 1080909.79


In [26]:
# Group by cuisine and compute average order value
cuisine_avg = (
    final_df.groupby("cuisine")["total_amount"]
    .mean()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

print(cuisine_avg)

# Top cuisine
top_cuisine = cuisine_avg.iloc[0]
print("Cuisine with highest average order value:", top_cuisine["cuisine"])
print("Average order value:", top_cuisine["total_amount"])


   cuisine  total_amount
3  Mexican    808.021344
2  Italian    799.448578
1   Indian    798.466011
0  Chinese    798.389020
Cuisine with highest average order value: Mexican
Average order value: 808.0213444401395


In [27]:
# Step 1: Calculate total order value per user
user_totals = (
    final_df.groupby("user_id")["total_amount"]
    .sum()
    .reset_index()
)

# Step 2: Filter users whose total > 1000
high_value_users = user_totals[user_totals["total_amount"] > 1000]

# Step 3: Count distinct users
count_users = high_value_users["user_id"].nunique()

print("Distinct users with total orders > ₹1000:", count_users)


Distinct users with total orders > ₹1000: 2544


In [28]:
import pandas as pd

# Define rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

# Create rating groups
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Group by rating range and sum revenue (fixed warning)
rating_revenue = (
    final_df.groupby("rating_range", observed=False)["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

print(rating_revenue)

# Highest range
top_range = rating_revenue.iloc[0]
print("Rating range with highest total revenue:", top_range["rating_range"])
print("Total revenue:", top_range["total_amount"])


  rating_range  total_amount
3    4.6 – 5.0    2197030.75
0    3.0 – 3.5    2136772.70
2    4.1 – 4.5    1960326.26
1    3.6 – 4.0    1717494.41
Rating range with highest total revenue: 4.6 – 5.0
Total revenue: 2197030.75


In [29]:
# Filter only Gold members
gold_df = final_df[final_df['membership'] == 'Gold']

# Group by city and calculate average order value
city_avg = (
    gold_df.groupby('city')['total_amount']
    .mean()
    .reset_index()
    .sort_values(by='total_amount', ascending=False)
)

print(city_avg)

# Top city
top_city = city_avg.iloc[0]
print("City with highest average order value among Gold members:", top_city['city'])
print("Average order value:", top_city['total_amount'])


        city  total_amount
1    Chennai    808.459080
2  Hyderabad    806.421034
0  Bangalore    793.223756
3       Pune    781.162243
City with highest average order value among Gold members: Chennai
Average order value: 808.4590800299178


In [30]:
# Group by cuisine to get:
# 1. Number of distinct restaurants
# 2. Total revenue

cuisine_stats = (
    final_df.groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .reset_index()
)

# Sort by distinct restaurants ascending (lowest first)
cuisine_stats_sorted = cuisine_stats.sort_values(
    by=["distinct_restaurants", "total_revenue"],
    ascending=[True, False]   # low restaurants but high revenue
)

print(cuisine_stats_sorted)

# Cuisine with lowest restaurants but good revenue
top_choice = cuisine_stats_sorted.iloc[0]
print("Cuisine with lowest distinct restaurants but significant revenue:",
      top_choice["cuisine"])
print("Distinct restaurants:", top_choice["distinct_restaurants"])
print("Total revenue:", top_choice["total_revenue"])


   cuisine  distinct_restaurants  total_revenue
0  Chinese                   120     1930504.65
2  Italian                   126     2024203.80
1   Indian                   126     1971412.58
3  Mexican                   128     2085503.09
Cuisine with lowest distinct restaurants but significant revenue: Chinese
Distinct restaurants: 120
Total revenue: 1930504.65


In [31]:
# Total orders
total_orders = final_df.shape[0]

# Gold member orders
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]

# Percentage
gold_percentage = round((gold_orders / total_orders) * 100)

print("Percentage of total orders placed by Gold members:", gold_percentage, "%")


Percentage of total orders placed by Gold members: 50 %


In [42]:
restaurant_stats = final_df.groupby('restaurant_name_x', observed=True).agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

top_restaurant = (
    restaurant_stats[restaurant_stats['total_orders'] < 20]
    .nlargest(1, 'avg_order_value')
)

top_restaurant[['restaurant_name_x', 'avg_order_value', 'total_orders']]


,restaurant_name_x,avg_order_value,total_orders
173,Hotel Dhaba Multicuisine,1040.222308,13


In [43]:
filtered = restaurant_stats[
    (restaurant_stats['total_orders'] < 20) &
    (restaurant_stats['restaurant_name_x'] != 'Hotel Dhaba Multicuisine')
]

filtered.nlargest(1, 'avg_order_value')


,restaurant_name_x,total_orders,avg_order_value
354,Sri Mess Punjabi,12,1029.180833


In [33]:
# Group by city and cuisine and sum revenue
combo_revenue = final_df.groupby(['city', 'cuisine'], observed=True)['total_amount'].sum().reset_index()

# Pick the top combination by revenue
top_combo = combo_revenue.nlargest(1, 'total_amount').iloc[0]

print("Combination contributing highest revenue:")
print("City:", top_combo['city'])
print("Cuisine:", top_combo['cuisine'])
print("Total Revenue:", top_combo['total_amount'])


Combination contributing highest revenue:
City: Bangalore
Cuisine: Mexican
Total Revenue: 571004.61


In [34]:
combo_revenue_sorted = (
    final_df.groupby(['membership', 'cuisine'], observed=True)
    .agg(total_revenue=('total_amount', 'sum'))
    .reset_index()
    .sort_values(by='total_revenue', ascending=False)
)

print("Revenue by membership + cuisine combination:")
print(combo_revenue_sorted)


Revenue by membership + cuisine combination:
  membership  cuisine  total_revenue
7    Regular  Mexican     1072943.30
6    Regular  Italian     1018424.75
3       Gold  Mexican     1012559.79
2       Gold  Italian     1005779.05
5    Regular   Indian      992100.27
1       Gold   Indian      979312.31
0       Gold  Chinese      977713.74
4    Regular  Chinese      952790.91


In [35]:
# Ensure correct date parsing for dd-mm-yyyy format
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# Extract quarter
final_df['quarter'] = final_df['order_date'].dt.quarter

# Sum revenue per quarter
revenue_by_quarter = final_df.groupby('quarter', observed=True)['total_amount'].sum()

# Map quarter numbers to readable names
quarter_mapping = {1: 'Q1 (Jan–Mar)', 2: 'Q2 (Apr–Jun)', 3: 'Q3 (Jul–Sep)', 4: 'Q4 (Oct–Dec)'}

# Find the top quarter
top_quarter = revenue_by_quarter.idxmax()

print("Revenue by quarter:")
print(revenue_by_quarter.sort_values(ascending=False))

print("Quarter with highest total revenue:", quarter_mapping[top_quarter])


Revenue by quarter:
quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64
Quarter with highest total revenue: Q3 (Jul–Sep)


In [36]:
# Filter orders by Gold members and count
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]

print("Total orders placed by Gold members:", gold_orders_count)



Total orders placed by Gold members: 4987


In [37]:
# Filter orders placed in Hyderabad
hyderabad_orders = final_df[final_df['city'] == 'Hyderabad']

# Sum total revenue and round to nearest integer
hyderabad_revenue = round(hyderabad_orders['total_amount'].sum())

print("Total revenue from Hyderabad:", hyderabad_revenue)


Total revenue from Hyderabad: 1889367


In [38]:
distinct_users = len(final_df['user_id'].unique())
print("Number of distinct users who placed at least one order:", distinct_users)


Number of distinct users who placed at least one order: 2883


In [39]:
avg_order_value = round(final_df.loc[final_df['membership'] == 'Gold', 'total_amount'].mean(), 2)
print("Average order value for Gold members:", avg_order_value)


Average order value for Gold members: 797.15


In [40]:
num_high_rating_orders = len(final_df[final_df['rating'] >= 4.5])
print("Number of orders for restaurants with rating ≥ 4.5:", num_high_rating_orders)


Number of orders for restaurants with rating ≥ 4.5: 3374


In [41]:
# Filter Gold member orders
gold_orders = final_df[final_df['membership'] == 'Gold']

# Group by city and sum revenue, then get top city
top_city_row = gold_orders.groupby('city', observed=True)['total_amount'].sum().nlargest(1)
top_city = top_city_row.index[0]

# Count orders in that city
num_orders_top_city = (gold_orders['city'] == top_city).sum()

print("Number of orders in top revenue city among Gold members:", num_orders_top_city)



Number of orders in top revenue city among Gold members: 1337


In [44]:
# Total number of rows in the final merged dataset
final_df.shape[0]


10000

In [45]:
orders_df.merge(users_df, on='user_id', how='left')


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold
